In [20]:
%pylab inline
import numpy as np
from scipy.integrate import odeint
from numpy.fft import fft, ifft, fftfreq
from time import time
import sys
import os
import torch
import torch.nn as nn
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'../..')))

from All import all0
from Select import select
from Train_fun import train_fun

torch.set_default_dtype(torch.float32)
device = torch.device('cuda:0')
np.set_printoptions(suppress=True)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


F:\anaconda\envs\pytorch\Lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['fft', 'select', 'time']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [21]:
def diff_AD(net,cor_train):
    cache = np.zeros((1,6))
    pts = cor_train.clone().detach().to(device)
    pts.requires_grad_(True)
    n_iter = len(pts)//10000
    for i in range(n_iter+1):
        pts0 = pts[10000*i:10000*(i+1)]
        outs = net(pts0)
        grad = torch.autograd.grad(outs, pts0, grad_outputs=torch.ones_like(outs), create_graph=True)[0]
        ut_t = (grad[:,0]).reshape(-1,1)
        ux_t = (grad[:,1]).reshape(-1,1)
        uxx_t  = ((torch.autograd.grad(ux_t, pts0, grad_outputs=torch.ones_like(outs), 
                create_graph=True)[0])[:,1]).reshape(-1,1)
        uxxx_t  = ((torch.autograd.grad(uxx_t, pts0, grad_outputs=torch.ones_like(outs), 
                create_graph=True)[0])[:,1]).reshape(-1,1)
        uxxxx_t  = ((torch.autograd.grad(uxxx_t, pts0, grad_outputs=torch.ones_like(outs), 
                create_graph=True)[0])[:,1]).reshape(-1,1)

        u_pred = outs.cpu().detach().numpy()
        ut = ut_t.cpu().detach().numpy()
        ux = ux_t.cpu().detach().numpy()
        uxx = uxx_t.cpu().detach().numpy()
        uxxx = uxxx_t.cpu().detach().numpy()
        uxxxx = uxxxx_t.cpu().detach().numpy()
        del ut_t, ux_t, uxx_t, uxxx_t, uxxxx_t, outs
        torch.cuda.empty_cache()
        cache = np.vstack((cache,np.hstack((u_pred,ux,uxx,uxxx,uxxxx,ut))))
    return cache[1:]

In [22]:
class FiveNet_sin(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.linear1 = nn.Linear(ni, 128)
        self.linear2 = nn.Linear(128,128)
        self.linear3 = nn.Linear(128, 64)
        self.linear4 = nn.Linear(64, 64)
        self.linear5 = nn.Linear(64,1)

    def forward(self, x):
        x = torch.sin(self.linear1(x))
        x = torch.sin(self.linear2(x))
        x = torch.sin(self.linear3(x))
        x = torch.sin(self.linear4(x))
        x = self.linear5(x)
        return x

In [23]:
def advection_rhs(u, t , params):
    
    k, L, x = params    
    kappa =  x
    deriv = ifft(1j*k*fft(kappa*u)) + 0.1*ifft(-k**2*fft(u)) 

    return real(deriv)

In [24]:
# Set size of grid
n = 501
m = 201
L = 5

# Set up grid
x = np.linspace(-L,L,n+1)[:-1];   dx = x[1]-x[0]
t = np.linspace(0,3,m);         dt = t[1]-t[0]
k = 2*np.pi*fftfreq(n, d = dx)

# Initial condition
np.random.seed(0)
u0 = np.cos(2*x*np.pi/L)
u01 = np.exp(-x**2)

# Solve with time dependent u_x term
params = (k,L,x)
u = odeint(advection_rhs, u0, t, args=(params,)).T
#u1 = odeint(advection_rhs, u01, t, args=(params,)).T
X, T = np.meshgrid(x, t)

In [25]:
data = np.hstack((T.reshape(-1,1), X.reshape(-1,1), u.T.reshape(-1,1))).astype(float32)
data = data[data[:,0]//0.015%4==1]
data = data[data[:,1]//0.02%6==1]

In [26]:
len(np.unique(data[:,0]))

54

In [27]:
t = data[:,0].reshape(-1,1)
x = data[:,1].reshape(-1,1)
u_clean = data[:,-1].reshape(-1,1)
per = 0.2
np.random.seed(0)
u_noise = u_clean + per*np.std(u_clean)*np.random.randn(u_clean.shape[0],u_clean.shape[1])
u_noise = u_noise.astype('float32')

In [28]:
cor_train = torch.from_numpy(data[:,:-1])
u_train = torch.from_numpy(u_noise).reshape(-1,1)
net = FiveNet_sin(2).to(device)
try:
    net.load_state_dict(torch.load('net/net_{}'.format(per)))
except:
    train_fun(net,cor_train,u_train,N_red_lr=4,epochs=3000,lr=0.001)
    torch.save(net.state_dict(),'net/net_{}'.format(per))

In [29]:
cache = diff_AD(net,cor_train)
t = data[:,0].reshape(-1,1)
x = data[:,1].reshape(-1,1)
u_pred = cache[:,0].reshape(-1,1)
ux = cache[:,1].reshape(-1,1)
uxx = cache[:,2].reshape(-1,1)
uxxx = cache[:,3].reshape(-1,1)
uxxxx = cache[:,4].reshape(-1,1)
ut = cache[:,5].reshape(-1,1)

In [30]:
rmse_loss(u_pred+x*ux+0.1*uxx,ut)

tensor(0.1624, device='cuda:0', dtype=torch.float64)

In [20]:
feature = np.hstack((t,x,u_pred,ux,uxx,uxxx,ut))
LC,exp_list,exp = all0('result_{}.txt'.format(per),feature,
                              state=3,alpha=10**-5,tol=0.05,
                              data_points=500,niterations=500)

/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/julia_helpers.py:162: UserWarning: `torch` was loaded before the Julia instance started. This may cause a segfault when running `PySRRegressor.fit`. To avoid this, please run `pysr.julia_helpers.init_julia()` *before* importing `torch`. For updates, see https://github.com/pytorch/pytorch/issues/78829
  warnings.warn(


Compiling Julia backend...


/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/julia_helpers.py:208: UserWarning: Your system's Python library is static (e.g., conda), so precompilation will be turned off. For a dynamic library, try using `pyenv` and installing with `--enable-shared`: https://github.com/pyenv/pyenv/blob/master/plugins/python-build/README.md#building-with---enable-shared.
  warnings.warn(


Started!

Expressions evaluated per second: 5.420e+05
Head worker occupation: 7.6%
Progress: 1231 / 7500 total iterations (16.413%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           3.271e+00  1.594e+01  y = 0.093842
3           1.461e-01  1.554e+00  y = (x₁ * x₃)
5           1.341e-01  4.299e-02  y = (x₃ * (x₁ * 0.94174))
7           1.182e-01  6.311e-02  y = ((x₃ * x₁) - (-0.029755 * x₄))
9           4.345e-03  1.652e+00  y = (((x₃ * x₁) + x₂) - (-0.10036 * x₄))
11          4.292e-03  6.203e-03  y = ((((x₃ - -0.0049911) * x₁) + x₂) - (-0.10036 * x₄))
15          4.267e-03  1.437e-03  y = (((x₃ * x₁) + x₂) - (-0.10036 * (x₄ + ((x₁ * -0.18184) * -...
                                  0.18184))))
17          4.252e-03  1.737e-03  y = (((x�� * x₁) + x₂) - (-0.10036 * (x₄ + (((x₁ * -0.15944) + ...
                                  -0.15944) * -0.18184))))
18          4.157

/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.668e+01, tolerance: 2.222e+00
  model = cd_fast.enet_coordinate_descent(
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.675e+01, tolerance: 2.222e+00
  model = cd_fast.enet_coordinate_descent(


...

[0.         0.         0.97991024 0.         0.         0.
 0.99907586 0.09892344 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]
[[ 0.          1.          0.65736     0.65736     0.00882891  0.00882891]
 [ 0.          2.          1.41070992  1.41070992  0.10996727  0.10996727]
 [ 0.          3.          0.33270111  0.33270111  0.02468847  0.02468847]
 [ 0.          4.          0.          0.          0.          0.        ]
 [ 1.          2.          0.02367564  2.44467684  0.14088748  0.11901388]
 [ 1.          3.          2.80728302  3.4377263   0.03450401  0.02151765]
 [ 1.          4.         13.99836631 13.99836631  0.00882891  0.00882891]
 [ 2.          3.          3.53552773  4.31261074  0.14587379  0.12716001]
 [ 2.          4.         15.01264697 15.01264697  0.10996727  0.10996727]
 [ 3.          4.          1.67949619  1.67949619  0.02468847  0.02468847]]
[[0.

/home/chr/code_11.6/Symmetry.py:196: RuntimeWarning: invalid value encountered in divide
  results[:, 2:] = results[:, 2:] / (np.sort(results[:, 2:])[:, 1].reshape(-1, 1))
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!

Expressions evaluated per second: 7.290e+05
Head worker occupation: 8.3%
Progress: 1559 / 7500 total iterations (20.787%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           3.843e-01  1.594e+01  y = x₁
3           3.594e-01  3.346e-02  y = (x₁ * 0.92065)
5           6.025e-03  2.044e+00  y = (x₁ + (0.10006 * x₂))
7           6.007e-03  1.503e-03  y = ((x₁ * 1.0022) + (0.10025 * x₂))
9           5.778e-03  1.941e-02  y = (x₁ - (-0.10024 * (x₂ - (-0.017573 / x₀))))
11          5.448e-03  2.944e-02  y = (x₁ + (0.10006 * (x₂ + ((-0.0015326 * x₃) / x₀))))
13          5.405e-03  3.905e-03  y = (x₁ + (0.10006 * (x₂ + ((-0.0015326 * (x₃ + -1.974)) / x₀)...
                                  )))
15          5.360e-03  4.230e-03  y = (x₁ - (-0.10006 * (x₂ + ((-0.0015326 * ((-0.10006 * x₃) * ...
                                  x₂)) / x₀))))
17          5.338e-03  2.045e-

/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.209e+03, tolerance: 2.222e+00
  model = cd_fast.enet_coordinate_descent(
/home/chr/code_11.6/Symmetry.py:196: RuntimeWarning: invalid value encountered in divide
  results[:, 2:] = results[:, 2:] / (np.sort(results[:, 2:])[:, 1].reshape(-1, 1))
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


[0.         0.         0.         0.99941907 0.10014731 0.
 0.         0.         0.         0.         0.         0.        ]
[[ 0.          1.          1.41493511  1.41493511  0.11908529  0.11908529]
 [ 0.          2.          0.33681724  0.33681724  0.02499392  0.02499392]
 [ 0.          3.          0.          0.          0.          0.        ]
 [ 1.          2.          3.53543451  4.32288114  0.15048527  0.11380396]
 [ 1.          3.         15.02154473 15.02154473  0.11908529  0.11908529]
 [ 2.          3.          1.70027467  1.70027467  0.02499392  0.02499392]]
[[0.         1.         0.11908529 1.         2.        ]
 [0.         2.         0.02499392 1.         2.        ]
 [0.         3.         0.                inf 0.        ]
 [1.         2.         0.11380396 0.75624654 3.        ]
 [1.         3.         0.11908529 1.         2.        ]
 [2.         3.         0.02499392 1.         2.        ]]




















found divide symmetry between 1 and 2

















In [22]:
k = select(LC,0.1)
exp_list[k]

[-1  1  1  1 -1 -1 -1 -1 -1]
[ 1  1  1  1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1 -1 -1]


x1 + 0.10006258*x2